# Utilisation de transformers pour la classification de textes

 > ℹ️ Inspiré de :
 > - https://github.com/nlp-with-transformers/notebooks/blob/main/02_classification.ipynb
 > - https://huggingface.co/docs/transformers/tasks/sequence_classification

🥅 **Objectifs**

- Savoir utiliser l'écosystème HuggingFace pour réutiliser des modèles pré-entraînés et les affiner sur de nouvelles données

🚨 **Consignes**

Les réponses aux questions doivent être données sur Moodle (Questionnaires "Réponses aux questions du TP"). Cela vous permettra d'obtenir un retour immédiat pour les questions fermées.

## 1. Installation des librairies nécessaires

In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1000"

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00


In [3]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.5 MB/s eta 0:00:00


In [4]:
! pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Features, Value, ClassLabel, Dataset, DatasetDict
import evaluate
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch

## 2. Récupération et préparation des données

In [6]:
!mkdir data
!wget -P data https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/winemag-fr_train.csv

mkdir: cannot create directory ‘data’: File exists
--2023-04-27 13:32:03--  https://git.unistra.fr/dbernhard/ftaa_data/-/raw/main/winemag-fr_train.csv
Resolving git.unistra.fr (git.unistra.fr)... 130.79.254.48
Connecting to git.unistra.fr (git.unistra.fr)|130.79.254.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144758 (2.0M) [text/plain]
Saving to: ‘data/winemag-fr_train.csv’

winemag-fr_train.cs 100%[===================>]   2.04M  1.98MB/s    in 1.0s    

2023-04-27 13:32:05 (1.98 MB/s) - ‘data/winemag-fr_train.csv’ saved [2144758/2144758]



In [7]:
wine_df = pd.read_csv("data/allocine_genres_train.csv", sep=",")
print(type(wine_df))
wine_df = wine_df.drop(columns=["acteur_1","acteur_2","acteur_3","allocine_id","annee_prod","annee_sortie","box_office_fr","couleur","duree","langues","nationalite","nb_critiques_presse","nb_critiques_spectateurs","nb_notes_spectateurs","note_presse","note_spectateurs","realisateurs","type_film"])
print(type(wine_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [8]:
# Liste de classes et ajout d'un identifiant numérique pour chaque classe
class_names = sorted(wine_df.genre.unique())
label2id = {class_names[i]:i for i in range(len(class_names))}
print(label2id)
id2label = {i:class_names[i] for i in range(len(class_names))}
print(id2label)

{'biopic': 0, 'comédie': 1, 'documentaire': 2, 'drame': 3, 'historique': 4, 'horreur': 5, 'policier': 6, 'romance': 7, 'science fiction': 8}
{0: 'biopic', 1: 'comédie', 2: 'documentaire', 3: 'drame', 4: 'historique', 5: 'horreur', 6: 'policier', 7: 'romance', 8: 'science fiction'}


In [9]:
# Paramètres

batch_size = 64
# Proportion des données qui sera utilisée
scale = 0.2

In [10]:
data_df = pd.DataFrame()
# Le texte décrivant chaque vin est composé des colonnes variety et description
split_titre = wine_df.titre.str.split('_')
data_df['text'] = split_titre.str.join(' ') + ' . ' + wine_df.synopsis
# La classe cible est la région (province) sous forme d'identifiant numérique
data_df['label'] = wine_df.genre.map(label2id)

# Transformation du DataFrame en objet de type Dataset utilisé par HuggingFace
film_features = Features({'text': Value('string'), 
                              'label': ClassLabel(names=class_names)})
data = Dataset.from_pandas(data_df, features=film_features)
# Découpage en train et test
data = data.train_test_split(test_size=0.2, shuffle=True, seed=12)

In [11]:
data['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['biopic', 'comédie', 'documentaire', 'drame', 'historique', 'horreur', 'policier', 'romance', 'science fiction'], id=None)}

In [12]:
data['train'][0]

{'text': "True Romance . Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .",
 'label': 7}

## 3. Tokénisation des données

Nous allons utiliser une variante de BERT (pour l'anglais) appelée DistilBERT. Ce modèle obtient des performances comparables à BERT, mais est de plus petite taille et plus rapide.

🚨 **DistilBERT est un modèle pour l'anglais et ne doit donc pas être utilisé pour des textes dans une autre langue. Pour rechercher des modèles adaptés à une autre langue, utiliser le filtre "Languages" sur https://huggingface.co/models** 🚨 

In [13]:
model_ckpt = "distilbert-base-multilingual-cased"
# Chargement du tokéniseur pré-entraîné correspondant au modèle utilisé
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [14]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [15]:
# Tokénisation des 2 premières instances
preprocess_function(data['train'][:2])

{'input_ids': [[101, 24079, 34404, 119, 10281, 16947, 10104, 10312, 77087, 117, 40653, 102204, 56900, 21702, 10109, 32650, 72384, 10253, 19866, 10260, 10119, 26552, 10221, 27249, 119, 13098, 14590, 10104, 12688, 16419, 10211, 18298, 92883, 119, 14214, 10231, 26642, 172, 112, 25205, 117, 19866, 34657, 12772, 169, 40653, 10608, 112, 11117, 169, 10845, 10110, 11329, 88601, 10112, 10248, 10141, 44979, 10104, 40653, 10986, 11135, 20042, 10138, 172, 112, 77087, 119, 10190, 10331, 10321, 22195, 10129, 10231, 95059, 71593, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 69783, 119, 69783, 117, 10208, 11744, 117, 17217, 11245, 10248, 10974, 32539, 13479, 117, 28281, 254, 10312, 28276, 10110, 22272, 10104, 10126, 97184, 97607, 33519, 10104, 10312, 263, 19224, 12131, 10104, 208, 85695, 220, 119, 14180, 171, 37925, 10165, 10141, 10347, 10350, 10

- Les 0 à la fin sont le résultat du padding (toutes les séquences du lot ont la même longueur)
- Les 0 dans le masque d'attention indiquent les tokens à ignorer dans le mécanisme d'attention (tokens ajoutés par le padding)

In [16]:
# Tokenisation de la totalité des données : chaque unité est remplacée par un identifiant numérique
tokenized_data = data.map(preprocess_function, batched=True, batch_size=1)

Map:   0%|          | 0/2300 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

In [17]:
tokenized_data['train'][0]

{'text': "True Romance . Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .",
 'label': 7,
 'input_ids': [101,
  24079,
  34404,
  119,
  10281,
  16947,
  10104,
  10312,
  77087,
  117,
  40653,
  102204,
  56900,
  21702,
  10109,
  32650,
  72384,
  10253,
  19866,
  10260,
  10119,
  26552,
  10221,
  27249,
  119,
  13098,
  14590,
  10104,
  12688,
  16419,
  10211,
  18298,
  92883,
  119,
  14214,
  10231,
  26642,
  172,
  112,
  25205,
  117,
  19866,
  34657,
  12772,
  169,
  40653,
  10608,
  112,
  11117,
  169,
  10845,
  10110,
  11329,
  88601,
  10112,
  10248,
  10141,
  44979,
  10104,
  40653,
  10986,
  11135,
  20042,
  10138,
  172,
  112,
  77087,
  119,
  10190,
  10331,
  10321,
  22195,
  10129,
  10231,
  

In [18]:
# Affichage des tokens. DistilBERT utilise l'algorithme WordPiece
tokens = tokenizer.convert_ids_to_tokens(tokenized_data['train'][0]['input_ids'])
print(tokenized_data['train'][0]['text'])
print(tokens)

True Romance . Le jour de son anniversaire , Clarence Worley rencontre la splendide Alabama dans un cinéma miteux . Coup de foudre immédiat . Après une nuit d' amour , Alabama avoue a Clarence qu' elle a été en fait engagée par le patron de Clarence comme cadeau d' anniversaire . De là va commencer une folle aventure .
['[CLS]', 'True', 'Romance', '.', 'Le', 'jour', 'de', 'son', 'anniversaire', ',', 'Clarence', 'Wo', '##rley', 'rencontre', 'la', 'sp', '##lendi', '##de', 'Alabama', 'dans', 'un', 'cinéma', 'mit', '##eux', '.', 'Co', '##up', 'de', 'fou', '##dre', 'im', '##mé', '##diat', '.', 'Après', 'une', 'nuit', 'd', "'", 'amour', ',', 'Alabama', 'avo', '##ue', 'a', 'Clarence', 'qu', "'", 'elle', 'a', 'été', 'en', 'fait', 'engagé', '##e', 'par', 'le', 'patron', 'de', 'Clarence', 'comme', 'ca', '##dea', '##u', 'd', "'", 'anniversaire', '.', 'De', 'là', 'va', 'commence', '##r', 'une', 'folle', 'aventure', '.', '[SEP]']




```
# Ce texte est au format code
```

- [CLS] et [SEP] indiquent le début et la fin de la séquence.
- Les tokens sont en minuscules.
- Le préfixe ## indique que le sous-mot n'est pas précédé par une espace

In [19]:
# Taille du vocabulaire
tokenizer.vocab_size

119547

In [20]:
# Taille de contexte maximum
tokenizer.model_max_length

512

## 4. Préparation de l'évaluation

In [21]:
accuracy = evaluate.load("accuracy")

In [22]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    return acc

## 5. Entraînement par affinage

On commence par charger le modèle pré-entraîné

In [23]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=len(class_names), id2label=id2label, label2id=label2id
).to(device)

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

In [24]:

training_args = TrainingArguments(
    output_dir=f"{model_ckpt}-finetuned-wine",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [25]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [26]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

❓ [1] Que constatez-vous par rapport aux résultats obtenus précédemment pour ce jeu de données ? Attention, ici nous ne faisons pas de validation croisée à 5 plis, les résultats sont évalués uniquement sur 20% des données (un seul pli).

## 6. Analyse des résultats

In [ ]:
# Prédictions pour les données de test
preds_output = trainer.predict(tokenized_data['test'])

In [ ]:
preds_output

In [ ]:
preds_output.metrics

---
Nous allons également afficher la matrice de confusion.





In [ ]:
y_preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
y_valid = tokenized_data['test']['label']

In [ ]:
labels = tokenized_data['test'].features['label'].names

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    labels_for_fig = [l[0:4]+'.' for l in labels]
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, 
                                  display_labels=labels_for_fig)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()
    
plot_confusion_matrix(y_preds, y_valid, labels)

❓ [2] Que constatez-vous ? Quelle est la classe pour laquelle les résultats sont les moins bons ? Pourquoi ?

Enfin, nous allons analyser les erreurs de classification. Pour cela, nous allons trier les instances par perte décroissante.

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # Fonction qui retourne la perte (entropie croisée) et la classe prédite
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}

    with torch.no_grad():
        output = model(**inputs)
        pred_label = torch.argmax(output.logits, axis=-1)
        loss = cross_entropy(output.logits, batch["label"].to(device), 
                             reduction="none")
    return {"loss": loss.cpu().numpy(), 
            "predicted_label": pred_label.cpu().numpy()}

In [ ]:
# Conversion des données au bon format
tokenized_data.set_format("torch", 
                            columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Calcul des valeurs de perte
tokenized_data["test"] = tokenized_data["test"].map(
    forward_pass_with_label, batched=True, batch_size=64)

In [ ]:
# Création d'un DataFrame avec les textes, les pertes les classe (prédites et attendues)

def label_int2str(row):
    return tokenized_data["train"].features["label"].int2str(row)

tokenized_data.set_format("pandas")
cols = ["text", "label", "predicted_label", "loss"]
df_test = tokenized_data["test"][:][cols]
df_test["label"] = df_test["label"].apply(label_int2str)
df_test["predicted_label"] = (df_test["predicted_label"]
                              .apply(label_int2str))

In [ ]:
# Pour éviter l'affichage tronqué des descriptions
pd.set_option('display.max_colwidth', -1)
# Affichage des 10 premières instances triées par perte décroissante
df_test.sort_values("loss", ascending=False).head(10)

In [ ]:
# Affichage des 10 premières instances triées par perte croissante
# Cela permet de voir les instances pour lesquelles les prédictions sont les plus certaines
df_test.sort_values("loss", ascending=True).head(10)